<a href="https://colab.research.google.com/github/owilli38/DSBA-6162/blob/main/DSBA6162_KNN_%26_Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import sklearn.metrics
#Preparing data
from sklearn import datasets

In [ ]:
dataset = pd.read_csv("AdultUCI.csv", header = None)
dataset.shape

(48843, 15)

In [ ]:
dataset.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
1,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,small
2,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,small
3,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,small
4,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,small


In [ ]:
for col in dataset.select_dtypes(include='object').columns:
    if dataset[col].nunique() > 32:
        dataset.drop(col, axis=1, inplace=True)
display(dataset.head())

,1,3,4,5,6,7,8,9,14
0,workclass,education,education-num,marital-status,occupation,relationship,race,sex,income
1,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,small
2,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,small
3,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,small
4,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,small


In [ ]:
dataset.isnull().any()

,0
1,True
3,False
4,False
5,False
6,True
7,False
8,False
9,False
14,True


In [ ]:
dataset.dropna(inplace=True)

In [ ]:
# The last column (index 14) is the target variable 'income'
# The first row is the header, so we drop it.
X = dataset.iloc[1:, :-1]  # Features are all columns except the last one
y = dataset.iloc[1:, -1]   # Target variable is the last column

print("Shape of features (X):", X.shape)
print("Shape of target variable (y):", y.shape)

Shape of features (X): (30718, 8)
Shape of target variable (y): (30718,)


In [ ]:
from sklearn.model_selection import train_test_split
pred_train, pred_test, tar_train, tar_test  =  train_test_split(X, y, test_size=.2, random_state = 2024)

In [ ]:
#Build Naive Bayes model on training data
from sklearn.naive_bayes import GaussianNB # Gaussian assumes normal distribution of the data
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np

# Identify categorical columns by their integer index after header removal
# Based on the dataset.head() output after dropping the header, the categorical columns are at indices 0, 1, 3, 4, 5, 6, 7
categorical_features_indices = [0, 1, 3, 4, 5, 6, 7]


# Create a column transformer for one-hot encoding
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features_indices)],
    remainder='passthrough') # Keep other columns (numerical) as they are

# Create a pipeline that first preprocesses the data and then applies the Gaussian Naive Bayes model
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', GaussianNB())])

# Fit the pipeline on the training data
gnb = pipeline.fit(pred_train, tar_train)

# Make predictions
gnb.predictions = gnb.predict(pred_test)

In [ ]:
sklearn.metrics.confusion_matrix(tar_test,gnb.predictions)

array([[1435,  107],
       [2465, 2137]])

In [ ]:
sklearn.metrics.accuracy_score(tar_test, gnb.predictions)

0.5813802083333334

In [ ]:
#Build KNN model
from sklearn import neighbors # object-oriented class
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Identify categorical columns by their integer index after header removal
# Based on the dataset.head() output after dropping the header, the categorical columns are at indices 0, 1, 3, 4, 5, 6, 7
categorical_features_indices = [0, 1, 3, 4, 5, 6, 7]

# Create a column transformer for one-hot encoding
preprocessor_knn = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features_indices)],
    remainder='passthrough') # Keep other columns (numerical) as they are

# Create a pipeline for KNN
pipeline_knn = Pipeline(steps=[('preprocessor', preprocessor_knn),
                              ('classifier', neighbors.KNeighborsClassifier(n_neighbors=5, weights='uniform'))])


# Fit the pipeline on the training data
knn = pipeline_knn.fit(pred_train,tar_train)
knn.predictions = knn.predict(pred_test)

In [ ]:
sklearn.metrics.confusion_matrix(tar_test,knn.predictions)

array([[ 783,  759],
       [ 455, 4147]])

In [ ]:
sklearn.metrics.accuracy_score(tar_test, knn.predictions)

0.8024088541666666

These accuracies between the two models--KNN and Naive Bayes Classifier--shows quite different results. The Naive Bayes Classifier does worse with just about 58% accuracy, while the KNN method of classification for this dataset achieved an accuracy of 80%, suggesting that the KNN method performs better with the test data--making the results more generalizable.